# Finite Precision Representations

In [ ]:
import numpy as np
from numpy import binary_repr

## Binary Representation

Humans typically use a base-10 numbering system, also known as the decimal system. This is likely because we have ten fingers. 

> An exception to this is the Mayans, who used a base-20 (vigesimal) system.

In the decimal system, each digit's position corresponds to a power of 10. For example, the number 1305 can be expressed as:

$1305_{10} = 5 \times 10^0 + 0 \times 10^1 + 3 \times 10^2 + 1 \times 10^3$

Computers, on the other hand, use a base-2 (binary) system, where each bit can be either 0 or 1. The same number, 1305, is written in binary as:

In [ ]:
binary_repr(1305)

We can verify this by converting the binary representation back to decimal:


$10100011001_2 = 1 \times 2^0 + 0 \times 2^1 + 0 \times 2^2 + 1 \times 2^3 + 1 \times 2^4 + $

$\quad 0 \times 2^5 + 0 \times 2^6 + 0 \times 2^7 + 1 \times 2^8 + 0 \times 2^9 + 1 \times 2^{10}$

$= 1305_{10}$

Binary representation can also include fractional parts, similar to a decimal point. For example, the decimal number $54.75_{10}$ can be written in binary.

In [ ]:
def decimal_to_binary(number):
    """Converts a decimal number to its binary representation."""
    # Convert the integer part to binary
    integer_part = binary_repr(int(number))

    # Convert the fractional part to binary
    fractional_part = number - int(number)
    binary_fractional_part = ""
    for i in range(20):
        fractional_part *= 2
        if fractional_part >= 1:
            binary_fractional_part += "1"
            fractional_part -= 1
        else:
            binary_fractional_part += "0"
        if fractional_part == 0:
            break

    # Combine the integer and fractional parts
    binary_representation = integer_part + "." + binary_fractional_part
    return binary_representation

print(f"54.75 in binary is: {decimal_to_binary(54.75)}")

Check:
$110110.11_2 = (1 \times 2^5 + 1 \times 2^4 + 0 \times 2^3 + 1 \times 2^2 + 1 \times 2^1 + 0 \times 2^0) + $

$\quad (1 \times 2^{-1} + 1 \times 2^{-2}) = 54.75_{10}$

### Example: Convert 0.1 to Binary

In [ ]:
print(f"0.1 in binary is: {decimal_to_binary(0.1)}")

The binary representation of 0.1 is a repeating fraction, which means it cannot be perfectly represented with a finite number of bits.

## Precision

Computers store data in units called *words*. The number of bits in a word determines the *precision*. The IEEE standard for floating-point arithmetic defines the following precisions:

| Precision | Number of Bits |
|-----------|----------------|
| Single    | 32             |
| Double    | 64             |
| Quad      | 128            |

For comparison, the binary representation of 1305 (`10100011001`) requires 11 bits. Modern computers typically use double precision for floating-point calculations, which is the default in Python 3.

## Integers

Integers are a fundamental data type for whole numbers and **do not suffer from round-off error**. However, they have a limited range based on the number of bits used for their storage.

The range of values an integer can store is determined by $2^{\text{bits}}$. For signed integers, this range is split between positive and negative values. Due to the representation of zero, the negative range is slightly larger than the positive range.

The minimum and maximum values for a signed integer are:

$min = -2^{\text{bits}-1}$

$max = 2^{\text{bits}-1} - 1$

> Modern computers use a method called *Two's Complement* to represent signed integers, rather than a simple sign bit.

### Example: What is the largest integer a double-precision variable can store?

In [ ]:
print(f"min: {-2**63}")
print(f"max: {2**63-1}")

print('\nCheck with the built-in numpy examiner:')
print(np.iinfo(np.int64))

### Example 2: Let's break it!

In [ ]:
# This works because 2**62 is within the range of a 64-bit integer
print(np.int64(2**62))

# This will cause an overflow error because 2**63 is too large
try:
    print(np.int64(2**63))
except OverflowError as e:
    print(e)

# This also works
print(np.int64(1000000000000000000))

# This will also cause an overflow error
try:
    print(np.int64(10000000000000000000))
except OverflowError as e:
    print(e)

## Floating-Point Numbers

Writing out very large numbers like $10,000,000,000,000,000,000$ is impractical. It is much more efficient to use scientific notation to represent the magnitude as an exponent:

$10,000,000,000,000,000,000 = 10^{19}$

### Floating-Point Decimal Numbers (Engineering Notation)

We can remove leading and placeholder trailing zeros by using a *floating point* to separate the fractional part (mantissa or significand) from the order of magnitude (exponent).

**Engineering Notation:** $mantissa \times 10^{exponent}$

| Decimal      | Engineering Notation     | Mantissa | Exponent |
|--------------|--------------------------|----------|----------|
| $265.73$     | $2.6573 \times 10^2$    | 2.6573   | 2        |
| $0.0001$     | $1 \times 10^{-4}$        | 1        | -4       |
| $-0.0034123$ | $-3.4123 \times 10^{-3}$ | -3.4123  | -3       |
| $1500^*$     | $1.5 \times 10^3$       | 1.5      | 3        |

*Assuming the trailing zeros are not significant.

**Note:**
1. The mantissa is a fraction. If we *normalize* it to have the decimal point after the first digit, we can represent it as an integer.
2. The exponent is the power of the number system's base (in this case, 10).

In [ ]:
def to_engineering_notation(number):
    """Converts a number to engineering notation."""
    if number == 0:
        return "0"

    exponent = 0
    while abs(number) < 1:
        number *= 10
        exponent -= 1
    while abs(number) >= 10:
        number /= 10
        exponent += 1

    return f"{number}E{exponent}"

print(to_engineering_notation(265.73))
print(to_engineering_notation(0.0001))
print(to_engineering_notation(-.0034123))
print(to_engineering_notation(1500))
print(to_engineering_notation(0))

### Floating-Point Binary Numbers

The same floating-point concept can be applied to binary numbers using base 2:

$mantissa \times 2^{exponent}$

#### Example: Convert 54.75 into Floating-Point Binary

$54.75_{10} = 110110.11_2$

To normalize this, we move the binary point so that it is after the first non-zero digit:

$1.1011011_2 \times 2^5$

The exponent is 5, which in binary is $101_2$. So the full floating-point representation is:

$1.1011011_2 \times 2^{101_2}$

## Precision in Floating-Point Numbers

If the mantissa and exponent had infinite range, we could represent all numbers perfectly. However, we are limited by the number of bits (precision). The bits are divided into three parts according to the IEEE 754 standard:

| Precision | Total Bits | Sign | Exponent | Mantissa |
|:----------|:-----------|:-----|:---------|:---------|
| Single    | 32         | 1    | 8        | 23       |
| Double    | 64         | 1    | 11       | 52       |
| Quad      | 128        | 1    | 15       | 112      |

The 'sign' bit determines whether the number is positive or negative. The 'exponent' bits store the exponent, and the 'mantissa' bits store the fractional part of the number.

### Example: How is 0.1 actually stored?

In [ ]:
print(format(0.1, '.55f'))

As you can see, the stored value is not exactly 0.1. This is because 0.1 is a repeating fraction in binary, so it cannot be perfectly represented. The difference between the true value and the stored value is the round-off error:

In [ ]:
error = 0.1 - float(format(0.1, '.55f'))
relative_error = error / 0.1
print(f"The relative error is: {relative_error}")

## Summary

In practice, we must be careful when performing arithmetic with floating-point numbers, especially when mixing numbers of different magnitudes or subtracting nearly equal numbers.

We cannot always rely on the associative property of addition:

In [ ]:
print(f"-1 + (1 + 1e-20) = {-1+(1+1e-20)}")
print(f"(-1 + 1) + 1e-20 = {(-1+1)+1e-20}")

Beware of **subtractive cancellation**!

In [ ]:
# Define two nearly equal numbers
a = np.float32(1.23456789)
b = np.float32(1.23456780)

# Perform subtraction
result = a - b

# Print the results with higher precision
print(f"a = {format(a, '.20f')}")
print(f"b = {format(b, '.20f')}")
print(f"a - b = {result}")